In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
%run ../tasking_manager_stats/data_management
%run ../tasking_manager_stats/map_tools

In [ ]:
project_id = 5889

In [ ]:
db = Database(project_id)

In [ ]:
start = db.get_start_date()
start

In [ ]:
nb_days = compute_nb_days(db, start)
nb_days

In [ ]:
day_duration = np.zeros(nb_days + 1)
cumulative_day_task = np.zeros(nb_days + 1)
unvalidated_tasks = []

In [ ]:
def add_task_validation_spent_time(task_data, day_duration):
    for task_hist in task_data['taskHistory']:
        if task_hist['action'] == 'LOCKED_FOR_VALIDATION':
            date = pd.to_datetime(task_hist['actionDate']).date()
            day = (date - start).days

            date_duration = pd.to_datetime(task_hist['actionText'])
            duration = date_duration.second + 60 * date_duration.minute + 3600 * date_duration.hour

            day_duration[day] += duration

In [ ]:
def add_task_validation_status(db, task_id, cumulative_day_task, unvalidated_tasks):
    
    task_validated = np.zeros(nb_days + 1)
    for task_hist in reversed(db.get_task_history()[str(task_id)]['taskHistory']):
        date = pd.to_datetime(task_hist['actionDate']).date()
        day = (date - start).days
        if task_hist['actionText'] == 'VALIDATED':
            task_validated[day:] = 1
    cumulative_day_task += task_validated
    
    if task_validated.max() == 0:
        unvalidated_tasks.append(task_id) 

In [ ]:
for task_id in db.get_task_ids():
    add_task_validation_spent_time(db.get_task_history()[str(task_id)], day_duration)
    add_task_validation_status(db, task_id, cumulative_day_task, unvalidated_tasks)

In [ ]:
day_duration

In [ ]:
cumulative_day_task

In [ ]:
unvalidated_tasks

In [ ]:
day_duration.sum() / 3600.0

In [ ]:
times = []
for day in range(nb_days + 1):
    times.append(start + pd.Timedelta(days=day))

In [ ]:
dpi = 50

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10), dpi=dpi, sharex=True)
ax.plot(times, day_duration / 3600.0, linewidth=2)
ax.set_title('Validation spent time in #' + str(project_id) + f' (total = {day_duration.sum() / 3600.0:.3} h)', fontsize=16)
ax.set_xlabel('Date', fontsize=16)
ax.set_ylabel('Time spent in hours', fontsize=16)
for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(16)
plt.savefig(str(project_id) + '_validation_spent_time', dpi=dpi)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10), dpi=dpi, sharex=True)
ax.plot(times, day_duration.cumsum() / 3600.0, linewidth=2)
ax.set_title('Cumulative validation spent time in #' + str(project_id) + f' (total = {day_duration.sum() / 3600.0:.3} h)', fontsize=16)
ax.set_xlabel('Date', fontsize=16)
ax.set_ylabel('Time spent in hours', fontsize=16)
for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(16)
plt.savefig(str(project_id) + '_cumulative_validation_spent_time', dpi=dpi)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10), dpi=dpi, sharex=True)
ax.plot(times, day_duration.cumsum() / 3600.0, linewidth=2)

ax2 = ax.twinx()
color = 'tab:green'
ax2.set_ylabel('Number of validated tasks', color=color)
ax2.plot(times, cumulative_day_task, color=color)
ax2.tick_params(axis='y', labelcolor=color)

title = 'Cumulative validation spent time in #' + str(project_id) + f' (total = {day_duration.sum() / 3600.0:.3} h)'
title = title + f'\n{int(cumulative_day_task[-1])} validated tasks on {len(db.get_task_ids())}'
ax.set_title(title)
ax.set_xlabel('Date')
ax.set_ylabel('Time spent in hours')
for item in ([ax.title, ax.xaxis.label, ax.yaxis.label, ax2.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels() +
             ax2.get_xticklabels() + ax2.get_yticklabels()):
    item.set_fontsize(16)
plt.savefig(str(project_id) + '_cumulative_validation_spent_time_and_validated_tasks', dpi=dpi)